In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [2]:
dataset = pd.read_csv("Train.csv")
finaltest = pd.read_csv("Test.csv")
riders = pd.read_csv('Riders.csv')
sample = pd.read_csv('SampleSubmission.csv')

In [4]:
placement = [col for col in dataset.columns if ((col[:len('Placement')] == 'Placement') or 
                                                (col[:len('Confirmation')] == 'Confirmation')) ]

In [5]:
placement

['Placement - Day of Month',
 'Placement - Weekday (Mo = 1)',
 'Placement - Time',
 'Confirmation - Day of Month',
 'Confirmation - Weekday (Mo = 1)',
 'Confirmation - Time']

In [6]:
xdata = dataset.drop(columns=['Time from Pickup to Arrival', 
                              'User Id', 'Order No', 
                              'Arrival at Destination - Day of Month', 
                              'Arrival at Destination - Time', 
                              'Arrival at Destination - Weekday (Mo = 1)'] + placement)
X = xdata.copy() 
X = X.merge(riders, how='left', on=['Rider Id']).drop(columns=['Rider Id'])

f_testx = finaltest.merge(riders, how='left', on=['Rider Id']).drop(columns=['User Id', 'Order No', 'Rider Id'] + placement)

In [31]:
def cleaner(input_df, nullthreshold=0.9, correlation_thresh=0.95, day_of_month_cols=[], day_of_week_cols=[]):
    input_dfc = input_df.copy()

    #########################################################################################
    # The Code below drops columns that have null values exceeding threshold and Columns that have ONLY one value
    input_dfc.drop(columns=['Vehicle Type', 'Precipitation in millimeters'], inplace=True)

    #########################################################################################

    #########################################################################################
    # This code converts time given by am and pm into seconds then applies cosine and sine
    def time_to_seconds(input_df):
        input_dfc = input_df.copy()

        from datetime import datetime

        for time_col in [col for col in input_df.columns if 'Time' in [col[-4:]]]:

            input_dfc[time_col + '_sin(seconds)'] = \
                input_df[time_col].apply(lambda time: np.sin(
                    (datetime.strptime(time, '%I:%M:%S %p') - datetime(1900, 1, 1)).total_seconds()
                    * (2. * np.pi / 86400)))  # there are 86400 seconds in a day

            input_dfc[time_col + '_cos(seconds)'] = \
                input_df[time_col].apply(lambda time: np.cos(
                    (datetime.strptime(time, '%I:%M:%S %p') - datetime(1900, 1, 1)).total_seconds()
                    * (2. * np.pi / 86400)))

            input_dfc.drop(columns=[time_col], inplace=True)

        return input_dfc

    input_dfc2 = time_to_seconds(input_dfc)
    #########################################################################################

    #########################################################################################
    # This code encodes ['Platform Type', 'Personal or Business']

    def one_encoder(input_df):
        for plat in ['Platform Type_2', 'Platform Type_3', 'Platform Type_4']:
            input_df[plat] = [1 if int(plat[-1]) == x else 0 for x in input_df['Platform Type']]
        
        input_df['Personal or Business_Personal'] = [1 if x == 'Personal' else 0 
                                             for x in input_df['Personal or Business']]
        
        
        return input_df.drop(columns=['Platform Type', 'Personal or Business'])
    
    input_dfc2 = one_encoder(input_dfc2)
    #########################################################################################

    def cyclic_days(input_df, month_days_cols, weekdays_cols):
        input_dfc = input_df.copy()

        for mday_col in month_days_cols:
            if mday_col in input_dfc.columns:

                input_dfc[mday_col + '_sin(day)'] = input_df[mday_col].apply(lambda day: np.sin(
                    day * (2. * np.pi / 31))
                )

                input_dfc[mday_col + '_cos(day)'] = input_df[mday_col].apply(lambda day: np.cos(
                    day * (2. * np.pi / 31))
                )

                input_dfc.drop(mday_col, inplace=True, axis=1)

        for wday_col in weekdays_cols:
            if wday_col in input_dfc.columns:

                input_dfc[wday_col + '_sin(day)'] = input_df[wday_col].apply(lambda day: np.sin(
                    day * (2. * np.pi / 7))
                )

                input_dfc[wday_col + '_cos(day)'] = input_df[wday_col].apply(lambda day: np.cos(
                    day * (2. * np.pi / 7))
                )

                input_dfc.drop(wday_col, inplace=True, axis=1)
            else:
                continue

        return input_dfc

    input_dfc2 = cyclic_days(input_dfc2, day_of_month_cols, day_of_week_cols)
    #########################################################################################

    input_dfc2.drop(columns=['Pickup - Time_sin(seconds)', 
                            'Pickup - Time_cos(seconds)', 
                            'Pickup - Day of Month_sin(day)', 
                            'Pickup - Day of Month_cos(day)', 
                            'Pickup - Weekday (Mo = 1)_sin(day)', 
                            'Pickup - Weekday (Mo = 1)_cos(day)'], inplace=True)
    #########################################################################################
    
    
    return input_dfc2

In [7]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21201 entries, 0 to 21200
Data columns (total 20 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Vehicle Type                          21201 non-null  object 
 1   Platform Type                         21201 non-null  int64  
 2   Personal or Business                  21201 non-null  object 
 3   Arrival at Pickup - Day of Month      21201 non-null  int64  
 4   Arrival at Pickup - Weekday (Mo = 1)  21201 non-null  int64  
 5   Arrival at Pickup - Time              21201 non-null  object 
 6   Pickup - Day of Month                 21201 non-null  int64  
 7   Pickup - Weekday (Mo = 1)             21201 non-null  int64  
 8   Pickup - Time                         21201 non-null  object 
 9   Distance (KM)                         21201 non-null  int64  
 10  Temperature                           16835 non-null  float64
 11  Precipitation i

In [8]:
input_df = X.copy()

In [15]:
for plat in ['Platform Type_2', 'Platform Type_3', 'Platform Type_4']:
            input_df[plat] = [1 if int(plat[-1]) == x else 0 for x in input_df['Platform Type']]
        
input_df['Personal or Business_Personal'] = [1 if x == 'Personal' else 0 
                                             for x in input_df['Personal or Business']]
input_df.drop(columns=['Platform Type', 'Personal or Business'])

,Vehicle Type,Arrival at Pickup - Day of Month,Arrival at Pickup - Weekday (Mo = 1),Arrival at Pickup - Time,Pickup - Day of Month,Pickup - Weekday (Mo = 1),Pickup - Time,Distance (KM),Temperature,Precipitation in millimeters,...,Destination Lat,Destination Long,No_Of_Orders,Age,Average_Rating,No_of_Ratings,Platform Type_2,Platform Type_3,Platform Type_4,Personal or Business_Personal
0,Bike,9,5,10:04:47 AM,9,5,10:27:30 AM,4,20.4,NaN,...,-1.300406,36.829741,1637,1309,13.8,549,0,1,0,0
1,Bike,12,5,11:40:22 AM,12,5,11:44:09 AM,16,26.4,NaN,...,-1.295004,36.814358,396,339,13.6,69,0,1,0,1
2,Bike,30,2,12:49:34 PM,30,2,12:53:03 PM,3,NaN,NaN,...,-1.300921,36.828195,1023,242,12.5,114,0,1,0,0
3,Bike,15,5,9:37:56 AM,15,5,9:43:06 AM,9,19.2,NaN,...,-1.257147,36.795063,886,283,14.5,113,0,1,0,0
4,Bike,13,1,10:03:53 AM,13,1,10:05:23 AM,9,15.4,NaN,...,-1.295041,36.809817,2311,872,14.1,533,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21196,Bike,20,3,3:58:49 PM,20,3,4:20:08 PM,3,28.6,NaN,...,-1.275285,36.802702,1270,367,14.4,131,0,1,0,1
21197,Bike,13,6,10:20:04 AM,13,6,10:33:27 AM,7,26.0,NaN,...,-1.331619,36.847976,1023,242,12.5,114,0,1,0,0
21198,Bike,7,4,5:30:17 PM,7,4,5:50:52 PM,20,29.2,NaN,...,-1.258414,36.804800,314,473,13.8,29,0,1,0,0
21199,Bike,4,3,9:38:59 AM,4,3,9:45:15 AM,13,15.0,NaN,...,-1.279209,36.794872,2451,1821,14.0,1101,0,0,0,1


In [17]:
len(input_df.index)

21201

In [27]:
Tes

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),...,Arrival at Destination - Time,Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival
0,Order_No_21660,User_Id_1329,Bike,3,Business,31,5,12:16:49 PM,31,5,...,NaN,-1.279518,36.823809,-1.273056,36.811298,Rider_Id_812,4402,1090,14.3,1301


In [28]:
Tes = pd.DataFrame([['Order_No_21660', 'User_Id_1329', 'Bike', 3, 'Business', 31, 5, '12:16:49 PM', 31, 5, '12:22:48 PM', 31, 5, '12:23:47 PM', 31, 5, '12:38:24 PM', 4, 21.8, np.nan, -1.2795183, 36.8238089, -1.273056, 36.811298, 'Rider_Id_812', 4402, 1090, 14.3, 1301]])

In [29]:
Tes.columns = finaltest.merge(riders, how='left', on=['Rider Id']).columns

In [30]:
Tes

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),...,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,No_Of_Orders,Age,Average_Rating,No_of_Ratings
0,Order_No_21660,User_Id_1329,Bike,3,Business,31,5,12:16:49 PM,31,5,...,NaN,-1.279518,36.823809,-1.273056,36.811298,Rider_Id_812,4402,1090,14.3,1301
